In [1]:
#8c80f14377224510b6a3c5d71525452d Client Id
#21ee065a096a4749b0bd051f07c4ca81 Client Secret
#0crxp0TRTMgfSjGURGcgyn5UAYtIaVZ29zXBL4h1C65zgqmOw_P4rldiHumJUvlk genius
#spotify:user:2c22r5e4gn78z1bvbfdo3sqbl user id
#spotify:playlist:0vopYysqPQ7IsQ9Z5X1uPv playlist id

In [2]:
from bs4 import BeautifulSoup
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dataclasses import dataclass
from typing import List

In [3]:
@dataclass
class MusicData:
    """Class for holding data of top music."""
    playlist: str
    track_names: List[str]
    track_artists: List[str]
    song_lyrics: List[str]

In [4]:
class GetLyrics():
    
    def __init__(self, spotify_client_id, spotify_client_secret, user_id, playlist_id, genius_key):
        self.spotify_client_id = spotify_client_id
        self.spotify_client_secret = spotify_client_secret
        self.user_id = user_id
        self.playlist_id = playlist_id
        self.genius_key = genius_key
        
    def get_playlist_info(self):
        token = SpotifyClientCredentials(client_id=self.spotify_client_id, client_secret=self.spotify_client_secret).get_access_token()
        sp = spotipy.Spotify(token)
        playlist = sp.user_playlist_tracks(self.user_id, self.playlist_id)
        self.playlist = playlist
        return self.playlist
    
    def get_track_names(self):
        track_names = []
        for song in range(len(self.playlist['items'])):
            track_names.append(self.playlist['items'][song]['track']['name'])
        self.track_names = track_names
        return self.track_names
    
    def get_track_artists(self):
        track_artists = []
        for song in range(len(self.playlist['items'])):
            track_artists.append(self.playlist['items'][song]['track']['artists'][0]['name'])
        self.track_artists = track_artists
        return self.track_artists
        
    def request_song_info(self, track_name, track_artist):
        self.track_name = track_name
        self.track_artist = track_artist
        base_url = 'https://api.genius.com'
        headers = {'Authorization': 'Bearer ' + self.genius_key}
        search_url = base_url + '/search'
        data = {'q': track_name + ' ' + track_artist}
        response = requests.get(search_url, data=data, headers=headers)
        self.response = response
        return self.response

    def check_hits(self):
        json = self.response.json()
        remote_song_info = None
        for hit in json['response']['hits']:
            if self.track_artist.lower() in hit['result']['primary_artist']['name'].lower():
                remote_song_info = hit
                break
        self.remote_song_info = remote_song_info
        return self.remote_song_info
    
    def get_url(self):
        song_url = self.remote_song_info['result']['url']
        self.song_url = song_url
        return self.song_url
    
    def scrape_lyrics(self):
        page = requests.get(self.song_url)
        html = BeautifulSoup(page.text, 'html.parser')
        lyrics1 = html.find("div", class_="lyrics")
        lyrics2 = html.find("div", class_="Lyrics__Container-sc-1ynbvzw-2 jgQsqn")
        if lyrics1:
            lyrics = lyrics1.get_text()
        elif lyrics2:
            lyrics = lyrics2.get_text()
        elif lyrics1 == lyrics2 == None:
            lyrics = None
        return lyrics

    def get_lyrics(self):
        playlist = GetLyrics.get_playlist_info(self)
        track_names = GetLyrics.get_track_names(self)
        track_artists = GetLyrics.get_track_artists(self)
        song_lyrics = []
        for i in range(len(self.track_names)):
            print("\n")
            print(f"Working on track {i}.")
            response = GetLyrics.request_song_info(self, self.track_names[i], self.track_artists[i])
            remote_song_info = GetLyrics.check_hits(self)
            if remote_song_info == None:
                lyrics = None
                print(f"Track {i} is not in the Genius database.")
            else:
                url = GetLyrics.get_url(self)
                lyrics = GetLyrics.scrape_lyrics(self)
                if lyrics == None:
                    print(f"Track {i} is not in the Genius database.")
                else:
                    print(f"Retrieved track {i} lyrics!")
            song_lyrics.append(lyrics)
        return MusicData(playlist, track_names, track_artists, song_lyrics)

In [5]:
spotify_client_id = "8c80f14377224510b6a3c5d71525452d"
spotify_client_secret = "21ee065a096a4749b0bd051f07c4ca81"
spotify_user_id = "2c22r5e4gn78z1bvbfdo3sqbl"
spotify_playlist_id = "0vopYysqPQ7IsQ9Z5X1uPv"
genius_key = "0crxp0TRTMgfSjGURGcgyn5UAYtIaVZ29zXBL4h1C65zgqmOw_P4rldiHumJUvlk"

In [6]:
songs = GetLyrics(spotify_client_id, spotify_client_secret, spotify_user_id, spotify_playlist_id, genius_key)
music_data = songs.get_lyrics()



Working on track 0.
Retrieved track 0 lyrics!


Working on track 1.
Retrieved track 1 lyrics!


Working on track 2.
Retrieved track 2 lyrics!


Working on track 3.
Retrieved track 3 lyrics!


Working on track 4.
Retrieved track 4 lyrics!


Working on track 5.
Retrieved track 5 lyrics!


Working on track 6.
Retrieved track 6 lyrics!


Working on track 7.
Retrieved track 7 lyrics!


Working on track 8.
Retrieved track 8 lyrics!


Working on track 9.
Retrieved track 9 lyrics!


Working on track 10.
Retrieved track 10 lyrics!


Working on track 11.
Retrieved track 11 lyrics!


Working on track 12.
Retrieved track 12 lyrics!


Working on track 13.
Retrieved track 13 lyrics!


Working on track 14.
Track 14 is not in the Genius database.


Working on track 15.
Track 15 is not in the Genius database.


Working on track 16.
Retrieved track 16 lyrics!


Working on track 17.
Retrieved track 17 lyrics!


Working on track 18.
Retrieved track 18 lyrics!


Working on track 19.
Retrieved track 19 ly

In [7]:
print(music_data.playlist)

{'href': 'https://api.spotify.com/v1/playlists/0vopYysqPQ7IsQ9Z5X1uPv/tracks?offset=0&limit=100&additional_types=track', 'items': [{'added_at': '2020-12-01T07:11:05Z', 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/2c22r5e4gn78z1bvbfdo3sqbl'}, 'href': 'https://api.spotify.com/v1/users/2c22r5e4gn78z1bvbfdo3sqbl', 'id': '2c22r5e4gn78z1bvbfdo3sqbl', 'type': 'user', 'uri': 'spotify:user:2c22r5e4gn78z1bvbfdo3sqbl'}, 'is_local': False, 'primary_color': None, 'track': {'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1Xyo4u8uXC1ZmMpatF05PJ'}, 'href': 'https://api.spotify.com/v1/artists/1Xyo4u8uXC1ZmMpatF05PJ', 'id': '1Xyo4u8uXC1ZmMpatF05PJ', 'name': 'The Weeknd', 'type': 'artist', 'uri': 'spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ'}], 'available_markets': ['AD', 'AE', 'AL', 'AR', 'AT', 'AU', 'BA', 'BE', 'BG', 'BH', 'BO', 'BR', 'BY', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK', 'DO', 'DZ', 'EC', 'EE', 'EG'

In [8]:
music_data.track_names

['Blinding Lights',
 'The Hills',
 'Starboy',
 'In Your Eyes',
 'Until I Bleed Out',
 'Heartless',
 'Faith',
 'Alone Again',
 'Too Late',
 'Scared To Live',
 'Escape From LA',
 'After Hours',
 'Save Your Tears',
 'Circles',
 'rockstar (feat. 21 Savage)',
 'Sunflower - Spider-Man: Into the Spider-Verse',
 'Wow.',
 'Better Now',
 'Congratulations',
 'I Fall Apart',
 'Come Get Her',
 'Black Beatles',
 'This Could Be Us',
 'No Type',
 'Swang',
 'Motorcycle Patches',
 'Saint',
 'Dubai Shit',
 'Black & Chinese',
 'Modern Slavery',
 'No Brainer',
 'Too Blessed',
 'Pussy Talk (feat. Quavo, Lil Wayne, & Jack Harlow) [Remix]',
 'FLIP THE SWITCH (feat. Drake)',
 "It Ain't Me (with Selena Gomez)",
 'Taki Taki (with Selena Gomez, Ozuna & Cardi B)',
 'Lose You To Love Me',
 "We Don't Talk Anymore (feat. Selena Gomez)",
 'Ice Cream (with Selena Gomez)',
 'Wolves',
 'Past Life (with Selena Gomez)',
 'Boyfriend',
 'Look At Her Now',
 'Rare',
 'Back To You - From 13 Reasons Why – Season 2 Soundtrack',
 

In [9]:
music_data.track_artists

['The Weeknd',
 'The Weeknd',
 'The Weeknd',
 'The Weeknd',
 'The Weeknd',
 'The Weeknd',
 'The Weeknd',
 'The Weeknd',
 'The Weeknd',
 'The Weeknd',
 'The Weeknd',
 'The Weeknd',
 'The Weeknd',
 'Post Malone',
 'Post Malone',
 'Post Malone',
 'Post Malone',
 'Post Malone',
 'Post Malone',
 'Post Malone',
 'Rae Sremmurd',
 'Rae Sremmurd',
 'Rae Sremmurd',
 'Rae Sremmurd',
 'Rae Sremmurd',
 'Huncho Jack',
 'Huncho Jack',
 'Huncho Jack',
 'Huncho Jack',
 'Huncho Jack',
 'DJ Khaled',
 'Rich The Kid',
 'City Girls',
 'Quavo',
 'Kygo',
 'DJ Snake',
 'Selena Gomez',
 'Charlie Puth',
 'BLACKPINK',
 'Selena Gomez',
 'Trevor Daniel',
 'Selena Gomez',
 'Selena Gomez',
 'Selena Gomez',
 'Selena Gomez',
 'Dr. Dre',
 'Eminem',
 'Eminem',
 'Eminem',
 'Eminem',
 'Eminem',
 'Eminem',
 'Eminem',
 'Eminem',
 'Eminem',
 'Eminem',
 'Eminem',
 'Eminem',
 'Eminem',
 'Eminem',
 'Eminem',
 'Eminem',
 'Eminem',
 'Eminem',
 'Eminem',
 'Eminem',
 'Juicy J',
 'N.W.A.',
 'J. Cole',
 'Waka Flocka Flame',
 '2Pac',
 

In [10]:
print(music_data.song_lyrics)

["[Intro]Yeah[Verse 1]I've been tryna callI've been on my own for long enoughMaybe you can show me\u2005how\u2005to love, maybeI'm\u2005going through withdrawalsYou don't even have\u2005to do too muchYou can turn me on with just a touch, baby[Pre-Chorus]I look around andSin City's cold and empty (Oh)No one's around to judge me (Oh)I can't see clearly when you're gone[Chorus]I said, ooh, I'm blinded by the lightsNo, I can't sleep until I feel your touchI said, ooh, I'm drowning in the nightOh, when I'm like this, you're the one I trustHey, hey, hey[Verse 2]I'm running out of time'Cause I can see the sun light up the skySo I hit the road in overdrive, baby, oh", '\n\n[Intro]\nYeah\nYeah\nYeah\n\n[Verse 1]\nYour man on the road, he doin\' promo\nYou said, "Keep our business on the low-low"\nI\'m just tryna get you out the friend zone\n\'Cause you look even better than the photos\nI can\'t find your house, send me the info\nDrivin\' through the gated residential\nFound out I was comin\', s

In [22]:
print(music_data.song_lyrics[98])

[Intro: 50 Cent]Yeah, uh-huhSo seductive[Chorus: 50 Cent & Olivia]I'll take you to the candy shopI'll let you lick the lollipopGo ahead, girl, don't you stopKeep goin' until you hit the spot, whoaI'll take you to the candy shop (Yeah)Want one taste of what I got? (Uh-huh)I'll have you spendin' all you got (Come on)Keep goin' until you hit the spot, whoa[Verse 1: 50 Cent]You could have it your way, how do you want it?You gon' back that thing up or should I push up on it?Temperature risin', okay, let's go to the next levelDance floor jam-packed, hot as a tea kettleI'll break it down for you now, baby, it's simpleIf you be a nympho, I be a nymphoIn the hotel or in the back of the rentalOn the beach or in the park, it's whatever you intoGot the magic stick, I'm the love doctorHave your friends teasin' you 'bout how sprung I got youWanna show me you could work it, baby? No problemGet on top, then get to bounce around like a low riderI'm a seasoned vet when it come to this shitAfter you work

In [ ]:
def remove_punc(self, str):
    punc = '''!()-[];:'",<>./?@#$%^&*_~'''
    for ele in str:  
        if ele in punc:  
            str = str.replace(ele, "")  
    return str

In [24]:
f = open("doc.py", "w", encoding='utf-8')
f.write("def getLyrics():\n")
f.write("\t lyrics = {\n")
id = 1
for i in range(100):
    if(music_data.song_lyrics[i] != None):
        f.write("\t\t" + str(i) + ": ")
        f.write("\t\t{ 'artist': '''" + music_data.track_artists[i]+"''',\n")
        f.write("\t\t 'id': " + str(i) + ",\n")
        f.write("\t\t  'title': '''" + music_data.track_names[i]+"''',\n")
        f.write("\t\t  'lyrics': '''" + music_data.song_lyrics[i] +"'''},\n")
f.write("\t }")

3